In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import re
sns.set()

In [ ]:
# pd.set_option('display.float', '{:.3f}'.format)
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [2]:
data_path = f"./data/loan_data_2007_2014.csv"
input_data = "test"

In [3]:
main_data = pd.read_csv(data_path)
data = main_data.copy() 

/tmp/ipykernel_348203/823853252.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  main_data = pd.read_csv(data_path)


In [ ]:
data.describe().transpose()

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.dtypes.value_counts()

In [ ]:
data.select_dtypes(include="object").columns.values

First lets confirm that our contunous varibles are numeric. Lets get rid of stirngs and alphanumeircs. Strip the number

In [ ]:
data.grade.unique()

In [ ]:
data.term.unique()

In [ ]:
def transform_term(val): return np.float32(re.search(r'\d+', str(val)).group()) if re.search(r'\d+', str(val)) else np.float32(0)

In [ ]:
data["term"] = data["term"].apply(transform_term)

In [ ]:
print(type(data.term[0]))
data.term.unique()

In [ ]:
data.emp_length.unique()

In [ ]:
def transform_empl_length(val):
    return np.float64(re.search(r'\d+', str(val)).group()) if re.search(r'\d+', str(val)) else np.float64(0)

In [ ]:
data["emp_length"] = data["emp_length"].apply(transform_empl_length)

In [ ]:
print(type(data["emp_length"][0]))
data.emp_length.unique()

earliest_cr_line: The month the borrower's earliest reported credit line was opened. Here we will count the number of months past since the earliest credit line was issued.

In [ ]:
data.earliest_cr_line

In [ ]:
data.earliest_cr_line.isna().sum()

In [ ]:
def transform_date_by_month(val):
    if pd.isna(val):
        return pd.NA  # Return NaN if the input is NaN
    else:
        converted = pd.to_datetime(val, format="%b-%y")
        months_difference = (pd.to_datetime("2023-11-01") - converted) / pd.Timedelta(4, 'W')
        rounded_months = round(pd.to_numeric(months_difference))
        return rounded_months


In [ ]:
data["earliest_cr_line_date"] = pd.to_datetime(data["earliest_cr_line"], format="%b-%y")

In [ ]:

data["months_since_earliest_cr_line"] = round(pd.to_numeric((pd.to_datetime("2023-11-01") - data["earliest_cr_line_date"])/ pd.Timedelta(4, 'W')))
data["months_since_earliest_cr_line"].describe()

We have detected the negative time since the earliest credit line date. Lets explore more.

In [ ]:
data["earliest_cr_line_date"].isna().sum()

In [ ]:
data.loc[:, ["earliest_cr_line", "earliest_cr_line_date", "months_since_earliest_cr_line"]][data["months_since_earliest_cr_line"] < 0]

We can see that many rows have future time time such as Sep-62. These variables are incorreectly converted into Sept 2062 - becuase the origin of the built-in time scale starts after 1970. We colud solve this issue by preprocessing the data or impute with maximum values of this variable. I will choose later.

In [ ]:
data.loc[data["months_since_earliest_cr_line"] < 0, "months_since_earliest_cr_line"] = data["months_since_earliest_cr_line"].max()
data["months_since_earliest_cr_line"].min()

Lets look at issued at date (issue_d).

In [ ]:
data.issue_d

In [ ]:
data["issue_d_date"] = pd.to_datetime(data["issue_d"], format="%b-%y")
data["months_since_issued_date"] = round(pd.to_numeric((pd.to_datetime("2023-11-01") - data["issue_d_date"])/ pd.Timedelta(4, "W")))
print(type(data["months_since_issued_date"][0]))
data["months_since_issued_date"].describe()

In [ ]:
objLs = data.select_dtypes(include="object").columns.values
for col in objLs:
    print(f"{data[col].value_counts()}\n==========================================================")

Lets explore discrete variables. I am going to use one-hot encoding for this exploration. The feature variable will increase by k-1 for each category in each variable. Variables such as:
- grade
- sub_grade
- home_ownership
- verification status 
- loan_status 
- purpose
- addr_state
- initial_list_status

In [ ]:
one_hot_data = []
one_hot_variables = ["grade", "home_ownership", "verification_status", "purpose", "addr_state", "initial_list_status"]

for category in one_hot_variables:
    one_hot_data.append(pd.get_dummies(data[category], prefix=category, prefix_sep=":"))

one_hot_data = pd.concat(one_hot_data, axis=1)
print(f"shape: {one_hot_data.shape}, type: {type(one_hot_data)}")

concatenate with dataframe.

In [ ]:
data =pd.concat([data, one_hot_data], axis=1)
print(f"new shape:{data.shape}")

Now, lets deal with missing values.

In [ ]:
data.isnull().sum().sort_values(ascending=False)

lets find out the missing number and drop the varibale id it is missing more than 70% of data.

In [ ]:
droppables = []
imputabels = []

for variable in data.columns.values:
    ratio = data[variable].isnull().sum() / data.shape[0]
    if ratio > 0.87: droppables.append(variable)
    if ratio > 0 and ratio < 0.87: imputabels.append(variable)

print(f"drops ({len(droppables)}): {droppables}\nimputables ({len(imputabels)}): {imputabels}")

In [ ]:
print(f"Before dropping cols: {data.shape}")
data.drop(columns=droppables, inplace=True)
print(f"After dropping cols: {data.shape}")

Total revolving high credit limit / credit limit, Failry, the missing values are equal to funded_amnt. Similarly, we have only 4 missing entries in annual_inc variable. It would be reasonable to fill with mean values,

In [ ]:
data['total_rev_hi_lim'].fillna(data['funded_amnt'], inplace=True)

In [ ]:
data["annual_inc"].describe()

In [ ]:
data["annual_inc"].fillna(data["annual_inc"].mean(), inplace=True)

In [ ]:
# picked numeric faetures 
cols_to_impute_with_zeros = ["months_since_earliest_cr_line", "mths_since_last_record", "acc_now_delinq", "total_acc", "pub_rec", "open_acc", "inq_last_6mths", "delinq_2yrs", "emp_length"]
for col in cols_to_impute_with_zeros:
    data[col].fillna(0, inplace=True)

In [ ]:
null_data = data.isnull().sum()
null_data = null_data.loc[null_data > 0].sort_values()

plt.bar(null_data.index, null_data,  color="blue")
plt.title("Null Values")
plt.xlabel("Variables")
plt.xticks(rotation=90)
plt.ylabel("Frequency")
plt.show()


Data Preparation

In [ ]:
plt.bar(data["loan_status"].value_counts().index, data["loan_status"].value_counts(),  color="blue")
plt.title("Loan Status Bar chart")
plt.xlabel("Loan Status")
plt.xticks(rotation=90)
plt.ylabel("Frequency")
plt.show()


Lets create a target variable. Here, I will create a good loan vs bad loan variable (good_bad). 1- Good (non-defaulters), 0 - Bad (defaulters)

In [ ]:
defaulter_label = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "Late (31-120 days)"]

data["good_bad"] = np.where(data["loan_status"].isin(defaulter_label), 0, 1)

In [ ]:
label_mapping = {0: 'Bad Loans', 1: 'Good Loans'}
temp_loan_data = data["good_bad"].map(label_mapping)

plt.pie(temp_loan_data.value_counts(), labels=temp_loan_data.value_counts().index,autopct='%2.2f%%', startangle=90, colors=['lightgreen', 'lightcoral'] )
plt.title("Target Data")
plt.show()

#### independent variables 

How will we categorise continious variables into categorical varibales. Binining, but how do we define the range ? Fine classing.

##### Fine Classing 
Fine classing, also known as fine binning, is a data preprocessing technique used in statistical analysis and machine learning to group continuous or numerical data into a larger number of smaller, more specific intervals or bins. This process is an extension of the binning or discretization process, which involves dividing a continuous variable into intervals or bins.

Fine classing aims to capture more detailed patterns and variations in the data by creating smaller intervals. This can be particularly useful when working with datasets that have a wide range or exhibit non-linear relationships.

The process of fine classing involves the following steps:

- Determine Number of Bins: Unlike coarse binning, where a smaller number of bins are used for a broader overview, fine classing involves choosing a larger number of bins to create more detailed intervals.

- Define Bin Boundaries: Establish the boundaries of each bin, ensuring that they are well-distributed across the range of the variable. The choice of bin boundaries can affect the interpretability and performance of the model.

- Assign Data Points to Bins: Place each data point into its corresponding bin based on the defined boundaries. This step is crucial for subsequent analysis or modeling.

Fine classing is often applied in various domains, including finance, marketing, and healthcare. In finance, for example, it might be used to create fine classes for credit score ranges to better assess the risk associated with different credit profiles. In marketing, it could be applied to customer age or income data for targeted campaign strategies. In healthcare, fine classing might be used to analyze patient data for personalized treatment plans.

The choice between fine classing and other binning techniques depends on the specific characteristics of the dataset and the goals of the analysis or modeling task. Fine classing may provide more detailed insights but may also lead to overfitting or increased computational complexity in certain situations. As with any preprocessing technique, it is important to carefully consider its implications for the downstream analysis or modeling.


Here, we are dealing with the categorical variables. we need to know the ability of each categories to predict the correct outcome. 

##### Weight of evidence 

Weight of Evidence (WoE) was developed primarily for the credit and financial industries to help build more predictive models to evaluate the risk of loan default. That is, to predict how likely the money lent to a person or institution is to be lost. Thus, Weight of Evidence is a measure of the "strength” of a grouping technique to separate good and bad risk (default).

- WoE will be 0 if the $P(Goods) / P(Bads) = 1$, that is, if the outcome is random for that group.
- If $P(Bads) > P(Goods)$ the odds ratio will be $< 1$ and,
- WoE will be $< 0$ if, $P(Goods) > P(Bads).$
  
WoE is well suited for Logistic Regression, because the Logit transformation is simply the log of the odds, i.e., 


$WoE_i$ : $ln(P(Goods)/P(Bads))$. The WoE are better far from zero. 


Suppose we have K categories, and we have computed WoE for each of the categories. 

##### Information value: 

Information Value (IV) is a measure commonly used in credit scoring and binary classification problems to assess the predictive power of an independent variable (feature) with respect to the target variable. It is particularly popular in the context of credit risk modeling.

\begin{equation}
    \sum_{i=1}^{k} (\%good - \% bad) \cdot \ln\left(\frac{\%good}{\%bad}\right)
\end{equation}

The Information Value provides a single, consolidated measure of the discriminatory power of the variable. Higher Information Values indicate stronger predictive power. Here's how Information Value is typically interpreted:

- $IV < 0.02 $ : No predictive power
- $0.02 < IV < 0.1 $ :  Weak predictive power
- $0.1 < IV < 0.3 $ : Medium predictive power
- $0.3 < IV < 0.5 $ :  Strong predictive power
- $IV > 0.5 $ :  Suspisciously high


- IV is based on the concept of entropy: It measures the reduction in entropy (uncertainty) about the target variable achieved by dividing the data into different categories based on the independent variable.

- It helps in feature selection: Variables with low or zero Information Value may be candidates for removal from the model, as they contribute little to the predictive power.

- Monotonicity: In practice, it's often desirable for the Information Value to exhibit a monotonic relationship with the predictive power. This means that as you move through the categories of a variable, the Information Value should generally increase or decrease.

*There are more advanced methods to use, as it uses WoE which leads to overfitting*


#### Data Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(['good_bad'], axis=1), data["good_bad"], test_size=0.2, random_state=45)
print(f"X tains shape: {x_train.shape},Y train shape: {y_train.shape}, test shape: {x_test.shape}, Y test shape: {y_test.shape}")

In [ ]:
if input_data == "train":
    x_train_prep = x_train.copy()
    y_train_prep = y_train.copy()
else:
    x_train_prep = x_test.copy()
    y_train_prep = y_test.copy()

In [ ]:
df_grade = pd.concat([x_train_prep['grade'], y_train_prep], axis=1)
df_grade.head()

##### WoE of Grade

In [ ]:
df_grade.groupby(df_grade.columns.values[0], as_index=False)[df_grade.columns.values[1]].count()

In [ ]:
df_grade.groupby(df_grade.columns.values[0], as_index=False)[df_grade.columns.values[1]].mean()

In [ ]:
df_grade = pd.concat([df_grade.groupby(df_grade.columns.values[0], as_index=False)[df_grade.columns.values[1]].count(),
                      df_grade.groupby(df_grade.columns.values[0], as_index=False)[df_grade.columns.values[1]].mean()], axis=1)

df_grade

In [ ]:
df_grade = df_grade.iloc[:, [0,1,3]]
df_grade.columns = [df_grade.columns.values[0], "n_observation", "prop_good"]
df_grade

In [ ]:
df_grade["prop_n_observation"] = df_grade["n_observation"]/df_grade["n_observation"].sum()
df_grade

In [ ]:
# count the number fof good borrowers and bad borrwers from each category 
df_grade["n_good"] = df_grade["prop_good"] * df_grade["n_observation"]
df_grade["n_bad"] = (1 - df_grade["prop_good"]) * df_grade["n_observation"]

# proportion of good and bad loans in each category 
df_grade["prop_n_good"] = df_grade["n_good"] / df_grade["n_good"].sum()
df_grade["prop_n_bad"] = df_grade["n_bad"] / df_grade["n_bad"].sum()

df_grade

In [ ]:
df_grade['woe'] = np.log(df_grade["prop_n_good"]/df_grade["prop_n_bad"])
df_grade = df_grade.sort_values(["woe"])
df_grade.reset_index(drop=True)
df_grade

In [ ]:
df_grade["diff_prop_good"] = df_grade["prop_good"].diff().abs()
df_grade["diff_woe"] = df_grade["woe"].diff().abs()
df_grade

In [ ]:
# lets calculate information value 
df_grade["IV"] = ((df_grade["prop_n_good"] - df_grade["prop_n_bad"]) * df_grade["woe"]).sum()
df_grade

Lets automate the process.

In [ ]:
def compute_woe_iv(df, variable_name, target_df):
    copy_df = pd.concat([df[variable_name], target_df], axis=1)
    copy_df = pd.concat([copy_df.groupby(variable_name, as_index=False)[copy_df.columns.values[1]].count(),
                         copy_df.groupby(copy_df[variable_name], as_index=False)[copy_df.columns.values[1]].mean()
                         ], axis=1)
    
    copy_df = copy_df.iloc[:, [0,1,3]]
    copy_df.columns = [copy_df.columns.values[0], "n_observation", "prop_good"]
    copy_df["prop_n_observation"] = copy_df["n_observation"] / copy_df["n_observation"].sum()
    # it returns the proprtion of good loans from df
    copy_df["n_good"] = copy_df["prop_good"] * copy_df["n_observation"]
    copy_df["n_bad"] = (1- copy_df["prop_good"]) * copy_df["n_observation"]
    
    copy_df["prop_n_good"] = copy_df["n_good"] / copy_df["n_good"].sum()
    copy_df["prop_n_bad"] = copy_df["n_bad"] / copy_df["n_bad"].sum()
    
    copy_df["woe"] = np.log(copy_df["prop_n_good"] / copy_df["prop_n_bad"])
    copy_df["iv"] = ((copy_df["prop_n_good"] - copy_df["prop_n_bad"]) * copy_df["woe"]).sum()
    
    copy_df = copy_df.sort_values(["woe"])
    copy_df = copy_df.reset_index(drop=True)
    return copy_df

In [ ]:
g_woe = compute_woe_iv(x_train_prep, "grade", y_train_prep)
g_woe

In [ ]:
def plot_woe(woe, x_tick_rotation=0):
    plt.figure(figsize=(15, 6))
    plt.plot(woe.iloc[:, 0].apply(str), woe['woe'], marker='o', linestyle='--', color='b', label=f'{woe.columns.values[0]} WoE Line chart')
    plt.xlabel(woe.columns.values[0])
    plt.xticks(rotation = x_tick_rotation)
    plt.ylabel("Weight of evidence")
    plt.title(f"Weight of evidence by {woe.columns.values[0]}")
    plt.legend()
    plt.show()

In [ ]:
plot_woe(g_woe)

The grades have monotonically increase from G(worse) to A(best). The greater the grade better the weight of evidence.

In [ ]:
list_of_ref_categories = []
list_of_dummy_variables = []
if input_data == "train":
    list_of_dummy_variables.extend([item for item in x_train_prep.columns.values if item.startswith('grade:')])
    list_of_ref_categories.append(list_of_dummy_variables[-1])
    print(f"{list_of_dummy_variables}\n{list_of_ref_categories}")

In [ ]:
home_ownership_woe = compute_woe_iv(x_train_prep, "home_ownership", y_train_prep)
home_ownership_woe

In [ ]:
plot_woe(home_ownership_woe)

Lookin at the Chart, The categories other, and None are associated with highest probability of default. WoE tells us that these categories have very few loans (< 0.1% each), and only one observation with ANY home ownership status. It does not make sense to have dummy varibles, neither to loose data. Hence, we combine such under represented category of similar nature. we will cobmined those one with riskiest variable Rent. Final dummy varibles [Rent, OWN,Mortgage ]

In [ ]:
x_train_prep["home_ownership:RENT_OTHER_NONE_ANY"] = sum([x_train_prep["home_ownership:RENT"], x_train_prep["home_ownership:NONE"], x_train_prep["home_ownership:OTHER"], x_train_prep["home_ownership:ANY"]])


In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["home_ownership:RENT_OTHER_NONE_ANY", "home_ownership:OWN", "home_ownership:MORTGAGE"])
    list_of_ref_categories.append("home_ownership:RENT_OTHER_NONE_ANY")
    print(f"{list_of_dummy_variables}\n{list_of_ref_categories}")

In [ ]:
addr_state_woe = compute_woe_iv(x_train_prep, "addr_state", y_train_prep)
addr_state_woe

In [ ]:
plot_woe(addr_state_woe, 90)

Looking at the graph, The NE, IA, and ID have very less observations, which has led them to have extreme WoE values. Lets cut them out and observe rest.

In [ ]:
# we do not see the north dakota here. We want to add if that is true
if "addr_state:ND" not in x_train_prep.columns.values: x_train_prep['addr_state:ND'] = 0

In [ ]:
plot_woe(addr_state_woe.iloc[3:-1,:], 90)

By this chart, we can categorise multiple states into one such as NE, IA, NV, AL, ID, ND (as we have no information about) FL, HI into one as they have similar woe. Similarly, WV, WY, DC, and ME in another categoruy, and CO, VT, MS, NH, AK, MT in another and similar for other states. we can notice that NY, CA, TX havehigher number of observation, hence we will leave them as they are.

In [ ]:
plot_woe(addr_state_woe.iloc[7:, :])

In [ ]:
x_train_prep["addr_state:NE_IA_NV_AL_ID_ND_FL_HI"] = sum([x_train_prep['addr_state:ND'], x_train_prep['addr_state:NE'],
                                                         x_train_prep['addr_state:IA'], x_train_prep['addr_state:NV'],
                                                         x_train_prep['addr_state:FL'], x_train_prep['addr_state:HI'],
                                                         x_train_prep['addr_state:AL'], x_train_prep['addr_state:ID']])


x_train_prep["addr_state:LA_MO_DE_MD"] = sum([x_train_prep['addr_state:LA'], x_train_prep['addr_state:MO'], 
                                              x_train_prep['addr_state:DE'], x_train_prep['addr_state:MD']])

x_train_prep['addr_state:NC_NM'] = sum([x_train_prep['addr_state:NC'], x_train_prep['addr_state:NM']])


x_train_prep['addr_state:NJ_AZ_VA_OK_TN'] = sum([x_train_prep['addr_state:NJ'], x_train_prep['addr_state:AZ'],
                                              x_train_prep['addr_state:VA'], x_train_prep['addr_state:OK'],
                                              x_train_prep['addr_state:TN']])

x_train_prep['addr_state:AR_PR_MI'] = sum([x_train_prep['addr_state:AR'], x_train_prep['addr_state:PA'],
                                              x_train_prep['addr_state:MI']])


x_train_prep['addr_state:UT_MA_RI_OH'] = sum([x_train_prep['addr_state:UT'], x_train_prep['addr_state:MA'],
                                              x_train_prep['addr_state:RI'], x_train_prep['addr_state:OH']])


x_train_prep['addr_state:KY_MN_GA_SD'] = sum([x_train_prep['addr_state:KY'], x_train_prep['addr_state:MN'],
                                              x_train_prep['addr_state:GA'], x_train_prep['addr_state:SD']])



x_train_prep['addr_state:IN_OR_WA_WI'] = sum([x_train_prep['addr_state:IN'], x_train_prep['addr_state:WA'],
                                              x_train_prep['addr_state:OR'], x_train_prep['addr_state:WI'] ])

x_train_prep['addr_state:IL_SC_CT_KS_CO'] = sum([x_train_prep['addr_state:IL'], x_train_prep['addr_state:SC'],
                                                 x_train_prep['addr_state:CT'], x_train_prep['addr_state:KS'],
                                                 x_train_prep['addr_state:CO']])



x_train_prep['addr_state:VT_MS_NH_AK_MT'] = sum([x_train_prep['addr_state:VT'], x_train_prep['addr_state:MS'],
                                                    x_train_prep['addr_state:NH'], x_train_prep['addr_state:AK'],
                                                    x_train_prep['addr_state:MT']])

x_train_prep['addr_state:WV_WY_DC_ME'] = sum([x_train_prep['addr_state:WV'], x_train_prep['addr_state:WY'],
                                              x_train_prep['addr_state:DC'], x_train_prep['addr_state:ME'] ])


In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(
        [   'addr_state:NY',
            'addr_state:CA',
            'addr_state:TX',
            'addr_state:NE_IA_NV_AL_ID_ND_FL_HI',
            'addr_state:LA_MO_DE_MD',
            'addr_state:NC_NM',
            'addr_state:NJ_AZ_VA_OK_TN',
            'addr_state:AR_PR_MI',
            'addr_state:UT_MA_RI_OH',
            'addr_state:KY_MN_GA_SD',
            'addr_state:IN_OR_WA_WI',
            'addr_state:IL_SC_CT_KS_CO',
            'addr_state:VT_MS_NH_AK_MT',
            'addr_state:WV_WY_DC_ME'
        ]
    )
    list_of_ref_categories.append('addr_state:NE_IA_NV_AL_ID_ND_FL_HI')

In [ ]:
verification_status_woe = compute_woe_iv(x_train_prep, "verification_status", y_train_prep)
verification_status_woe

In [ ]:
plot_woe(verification_status_woe)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend([item  for item in x_train_prep.columns.values if item.startswith("verification_status:")])
    list_of_ref_categories.append(list_of_dummy_variables[-1])

In [ ]:
purpose_woe = compute_woe_iv(x_train_prep, "purpose", y_train_prep)
purpose_woe

In [ ]:
plot_woe(purpose_woe, 90)

In [ ]:
x_train_prep['purpose:educ_small_biz_wedd_renno_enerby_moving_other_house'] = sum([x_train_prep['purpose:educational'], x_train_prep['purpose:small_business'],
                                                                 x_train_prep['purpose:wedding'], x_train_prep['purpose:renewable_energy'],
                                                                 x_train_prep['purpose:moving'], x_train_prep['purpose:house'],
                                                                  x_train_prep['purpose:other']])
x_train_prep["purpose:medical_vacation_wedding"] = sum([x_train_prep["purpose:medical"], x_train_prep["purpose:vacation"], x_train_prep["purpose:wedding"]])

x_train_prep["purpose:major_purchase_home_imp_car"]  = sum([x_train_prep["purpose:major_purchase"], x_train_prep["purpose:home_improvement"],
                                                            x_train_prep["purpose:car"]])

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["purpose:educ_small_biz_wedd_renno_enerby_moving_other_house","purpose:medical_vacation_wedding", 
                                    "purpose:major_purchase_home_imp_car", "purpose:credit_card", "purpose:debt_consolidation"])
    list_of_ref_categories.append("purpose:educ_small_biz_wedd_renno_enerby_moving_other_house")

In [ ]:
initial_list_status_we = compute_woe_iv(x_train_prep, "initial_list_status", y_train_prep)
initial_list_status_we

In [ ]:
plot_woe(initial_list_status_we)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["initial_list_status:f", "initial_list_status:w"])
    list_of_ref_categories.append("initial_list_status:f")

Lets do for continous variables.

In [ ]:
def compute_woe_iv_continous(df, variable_name, target_df):
    copy_df = pd.concat([df[variable_name], target_df], axis=1)
    copy_df = pd.concat([copy_df.groupby(variable_name, as_index=False)[copy_df.columns.values[1]].count(),
                         copy_df.groupby(copy_df[variable_name], as_index=False)[copy_df.columns.values[1]].mean()
                         ], axis=1)
    
    copy_df = copy_df.iloc[:, [0,1,3]]
    copy_df.columns = [copy_df.columns.values[0], "n_observation", "prop_good"]
    copy_df["prop_n_observation"] = copy_df["n_observation"] / copy_df["n_observation"].sum()
    # it returns the proprtion of good loans from df
    copy_df["n_good"] = copy_df["prop_good"] * copy_df["n_observation"]
    copy_df["n_bad"] = (1- copy_df["prop_good"]) * copy_df["n_observation"]
    
    copy_df["prop_n_good"] = copy_df["n_good"] / copy_df["n_good"].sum()
    copy_df["prop_n_bad"] = copy_df["n_bad"] / copy_df["n_bad"].sum()
    
    copy_df["woe"] = np.log(copy_df["prop_n_good"] / copy_df["prop_n_bad"])
    copy_df["iv"] = ((copy_df["prop_n_good"] - copy_df["prop_n_bad"]) * copy_df["woe"]).sum()
    
    # copy_df = copy_df.sort_values(["woe"])
    # copy_df = copy_df.reset_index(drop=True)
    return copy_df

In [ ]:
term_woe = compute_woe_iv_continous(x_train_prep, "term", y_train_prep)
term_woe

In [ ]:
plot_woe(term_woe)

In [ ]:
x_train_prep["term:36"] = np.where((x_train_prep["term"] == 36), 1, 0)
x_train_prep["term:60"] = np.where((x_train_prep["term"] == 60), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["term:36", "term:60"])
    list_of_ref_categories.append("term:60")

In [ ]:
emp_length_woe = compute_woe_iv_continous(x_train_prep, "emp_length", y_train)
emp_length_woe

In [ ]:
plot_woe(emp_length_woe)

In [ ]:
x_train_prep["emp_length:0"] = np.where((x_train_prep["emp_length"] == 0), 1, 0)
x_train_prep["emp_length:1"] = np.where((x_train_prep["emp_length"] == 1), 1, 0)
x_train_prep["emp_length:2_4"] = np.where((x_train_prep["emp_length"].isin(range(2,5))), 1, 0)
x_train_prep["emp_length:5_6"] = np.where((x_train_prep["emp_length"].isin(range(5,7))), 1, 0)
x_train_prep["emp_length:7_9"] = np.where((x_train_prep["emp_length"].isin(range(7, 10))), 1, 0)
x_train_prep["emp_length:10"] = np.where((x_train_prep["emp_length"] == 10), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["emp_length:0", "emp_length:1", "emp_length:2_4", "emp_length:5_6", "emp_length:7_9", "emp_length:10"])
    list_of_ref_categories.append("emp_length:0")

In [ ]:
x_train_prep["months_since_issued_date"].unique()

In [ ]:
# Lets do fine and coarse classing 
x_train_prep["months_since_issued_date_factor"] = pd.cut(x_train_prep["months_since_issued_date"], 50)

In [ ]:
cat_months_since_issued_date_factor_woe = compute_woe_iv_continous(x_train_prep, "months_since_issued_date_factor", y_train_prep)
cat_months_since_issued_date_factor_woe

In [ ]:
plot_woe(cat_months_since_issued_date_factor_woe, 90)

In [ ]:
x_train_prep["months_since_issued_date:<117"] = np.where(x_train_prep["months_since_issued_date"].isin(range(117)), 1,0)
x_train_prep["months_since_issued_date:117_119"] = np.where(x_train_prep["months_since_issued_date"].isin(range(117,119)), 1,0)
x_train_prep["months_since_issued_date:119_123"] = np.where(x_train_prep["months_since_issued_date"].isin(range(119, 123)), 1,0)
x_train_prep["months_since_issued_date:123_131"] = np.where(x_train_prep["months_since_issued_date"].isin(range(123, 131)), 1,0)
x_train_prep["months_since_issued_date:131_149"] = np.where(x_train_prep["months_since_issued_date"].isin(range(131, 149)), 1,0)
x_train_prep["months_since_issued_date:149_172"] = np.where(x_train_prep["months_since_issued_date"].isin(range(149, 172)), 1,0)
x_train_prep["months_since_issued_date:>172"] = np.where(x_train_prep["months_since_issued_date"].isin(range(172, int(x_train_prep["months_since_issued_date"].max()))), 1,0)


In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["months_since_issued_date:<117", "months_since_issued_date:117_119", "months_since_issued_date:119_123", 
                                "months_since_issued_date:123_131", "months_since_issued_date:131_149", "months_since_issued_date:149_172", "months_since_issued_date:>172"])
    list_of_ref_categories.append("months_since_issued_date:>172")

In [ ]:
x_train_prep["int_rate_factor"] = pd.cut(x_train_prep["int_rate"], 50)

int_rate_factor_woe = compute_woe_iv_continous(x_train_prep, "int_rate_factor", y_train_prep)
int_rate_factor_woe

In [ ]:
plot_woe(int_rate_factor_woe, 90)

In [ ]:
x_train_prep["int_rate:<9.54"] = np.where((x_train_prep["int_rate"]<= 9.548), 1, 0)
x_train_prep["int_rate:9.54_12.025"] = np.where((x_train_prep["int_rate"]> 9.548) & (x_train_prep["int_rate"]<=12.025), 1, 0)
x_train_prep["int_rate:12.025_15.74"] = np.where((x_train_prep["int_rate"]> 12.025) & (x_train_prep["int_rate"]<=15.74), 1, 0)
x_train_prep["int_rate:15.74_20.281"] = np.where((x_train_prep["int_rate"]> 15.74) & (x_train_prep["int_rate"]<=20.281), 1, 0)
x_train_prep["int_rate:>20.281"] = np.where((x_train_prep["int_rate"] > 20.281), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["int_rate:<9.54", "int_rate:9.54_12.025", "int_rate:12.025_15.74", 
                                "int_rate:15.74_20.281", "int_rate:>20.281"])
    list_of_ref_categories.append("int_rate:>20.281")

In [ ]:
x_train_prep["funded_amnt_factor"] = pd.cut(x_train_prep["funded_amnt"], 50)
funded_amount_woe = compute_woe_iv_continous(x_train_prep, "funded_amnt_factor", y_train_prep)
funded_amount_woe

In [ ]:
plot_woe(funded_amount_woe, 90)

The chart shows very different WoE on each category. It appears like trens, and almost horizontal. We can not find the pattern between indepndent and dependent variable. Hence, we will not use this varaible as predicator.

In [ ]:
x_train_prep["annual_inc_factor"] = pd.cut(x_train_prep["annual_inc"], 105)
annual_inc_woe = compute_woe_iv_continous(x_train_prep, "annual_inc_factor", y_train_prep)
annual_inc_woe

Majority of annual income samples are from < 144000. Here, we will split the data < 144000 and > 144000, and further categorise by usng WoE method.

In [ ]:
annual_income_df = x_train_prep.loc[x_train_prep["annual_inc"] <= 144000, :]

In [ ]:
annual_income_df["annual_inc_factor"] = pd.cut(annual_income_df["annual_inc"], 50)
annual_inc_temp_woe = compute_woe_iv_continous(annual_income_df, "annual_inc_factor", y_train_prep[annual_income_df.index])
annual_inc_temp_woe

In [ ]:
plot_woe(annual_inc_temp_woe, 90)

In [ ]:
x_train_prep["annual_inc:<20K"] = np.where((x_train_prep["annual_inc"] <= 20000), 1, 0)
x_train_prep["annual_inc:<20K_30K"] = np.where((x_train_prep["annual_inc"] > 20000) & (x_train_prep["annual_inc"] >= 30000) , 1, 0)
x_train_prep["annual_inc:<30K_40K"] = np.where((x_train_prep["annual_inc"] > 30000) & (x_train_prep["annual_inc"] <= 40000) , 1, 0)
x_train_prep["annual_inc:<40K_50K"] = np.where((x_train_prep["annual_inc"] > 40000) & (x_train_prep["annual_inc"] <= 50000) , 1, 0)
x_train_prep["annual_inc:<50K_60K"] = np.where((x_train_prep["annual_inc"] > 50000) & (x_train_prep["annual_inc"] <= 60000) , 1, 0)
x_train_prep["annual_inc:<60K_70K"] = np.where((x_train_prep["annual_inc"] > 60000) & (x_train_prep["annual_inc"] <= 70000) , 1, 0)
x_train_prep["annual_inc:<70K_80K"] = np.where((x_train_prep["annual_inc"] > 70000) & (x_train_prep["annual_inc"] <= 80000) , 1, 0)
x_train_prep["annual_inc:<80K_90K"] = np.where((x_train_prep["annual_inc"] > 80000) & (x_train_prep["annual_inc"] <= 90000) , 1, 0)
x_train_prep["annual_inc:<90K_100K"] = np.where((x_train_prep["annual_inc"] > 90000) & (x_train_prep["annual_inc"] <= 100000) , 1, 0)
x_train_prep["annual_inc:<100K_120K"] = np.where((x_train_prep["annual_inc"] > 100000) & (x_train_prep["annual_inc"] <= 120000) , 1, 0)
x_train_prep["annual_inc:<120K_140K"] = np.where((x_train_prep["annual_inc"] > 120000) & (x_train_prep["annual_inc"] <= 140000) , 1, 0)
x_train_prep["annual_inc:>140K"] = np.where((x_train_prep["annual_inc"] > 140000), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["annual_inc:<20K", "annual_inc:<20K_30K", "annual_inc:<30K_40K", "annual_inc:<40K_50K", "annual_inc:<50K_60K", "annual_inc:<60K_70K",
                                "annual_inc:<70K_80K", "annual_inc:<80K_90K", "annual_inc:<90K_100K", "annual_inc:<100K_120K", "annual_inc:<120K_140K", "annual_inc:>140K"])
    list_of_ref_categories.append("annual_inc:<20K")

In [ ]:
mths_since_last_delinq_df = x_train_prep[pd.notnull(x_train_prep["mths_since_last_delinq"])]
mths_since_last_delinq_df['mths_since_last_delinq_factor'] = pd.cut(mths_since_last_delinq_df["mths_since_last_delinq"], 50)
mths_since_last_delinq_woe = compute_woe_iv_continous(mths_since_last_delinq_df, "mths_since_last_delinq_factor", y_train_prep[mths_since_last_delinq_df.index])
mths_since_last_delinq_woe

In [ ]:
plot_woe(mths_since_last_delinq_woe, 90)

In [ ]:
x_train_prep["mths_since_last_delinq:0_3"] = np.where((x_train_prep["mths_since_last_delinq"] >= 0 & (x_train_prep["mths_since_last_delinq"] < 4)), 1, 0)
x_train_prep["mths_since_last_delinq:4_37"] = np.where((x_train_prep["mths_since_last_delinq"] >= 4 & (x_train_prep["mths_since_last_delinq"] < 37)), 1, 0)
x_train_prep["mths_since_last_delinq:37_56"] = np.where((x_train_prep["mths_since_last_delinq"] >= 37 & (x_train_prep["mths_since_last_delinq"] < 56)), 1, 0)
x_train_prep["mths_since_last_delinq:>56"] = np.where((x_train_prep["mths_since_last_delinq"] >= 56), 1, 0)
x_train_prep["mths_since_last_delinq:missing"] = np.where((x_train_prep["mths_since_last_delinq"].isnull()), 1, 0)

In [ ]:
list_of_dummy_variables.extend(["mths_since_last_delinq:0_3", "mths_since_last_delinq:4_37", "mths_since_last_delinq:37_56", "mths_since_last_delinq:>56", "mths_since_last_delinq:missing"])
list_of_ref_categories.append("mths_since_last_delinq:0_3")

In [ ]:
x_train_prep["months_since_earliest_cr_line_factor"] = pd.cut(x_train_prep["months_since_earliest_cr_line"], 50)
months_since_earliest_cr_line_factor_woe = compute_woe_iv_continous(x_train_prep, "months_since_earliest_cr_line_factor", y_train_prep)
months_since_earliest_cr_line_factor_woe

In [ ]:
plot_woe(months_since_earliest_cr_line_factor_woe, 90)

In [ ]:
plot_woe(months_since_earliest_cr_line_factor_woe.iloc[7: , : ], 90)

In [ ]:
x_train_prep['months_since_earliest_cr_line:<171'] = np.where(x_train_prep['months_since_earliest_cr_line'].isin(range(171)), 1, 0)
x_train_prep['months_since_earliest_cr_line:171_228'] = np.where(x_train_prep['months_since_earliest_cr_line'].isin(range(171, 228)), 1, 0)
x_train_prep['months_since_earliest_cr_line:228_257'] = np.where(x_train_prep['months_since_earliest_cr_line'].isin(range(228, 257)), 1, 0)
x_train_prep['months_since_earliest_cr_line:257_371'] = np.where(x_train_prep['months_since_earliest_cr_line'].isin(range(257, 371)), 1, 0)
x_train_prep['months_since_earliest_cr_line:371_471'] = np.where(x_train_prep['months_since_earliest_cr_line'].isin(range(371, 471)), 1, 0)
x_train_prep['months_since_earliest_cr_line:>486'] = np.where(x_train_prep['months_since_earliest_cr_line'].isin(range(486, int(x_train_prep['months_since_earliest_cr_line'].max()))), 1, 0)

In [ ]:
list_of_dummy_variables.extend(["months_since_earliest_cr_line:<171", "months_since_earliest_cr_line:171_228", "months_since_earliest_cr_line:228_257",
                                "months_since_earliest_cr_line:257_371", "months_since_earliest_cr_line:371_471", "months_since_earliest_cr_line:>486"])
list_of_ref_categories.append("months_since_earliest_cr_line:<171")

In [ ]:
x_train_prep["installment_factor"] = pd.cut(x_train_prep['installment'], 50)
installment_woe = compute_woe_iv_continous(x_train_prep, "installment_factor", y_train_prep)
installment_woe

In [ ]:
plot_woe(installment_woe, 90)

This feature does not contribute much to dependent variable. Hence, we omit this.

In [ ]:
delinq_2years_woe = compute_woe_iv_continous(x_train_prep, "delinq_2yrs", y_train_prep)
delinq_2years_woe

In [ ]:
plot_woe(delinq_2years_woe, 90)

In [ ]:
x_train_prep['delinq_2yrs:0'] = np.where((x_train_prep['delinq_2yrs'] == 0), 1, 0)
x_train_prep['delinq_2yrs:1_3'] = np.where((x_train_prep['delinq_2yrs'] >= 1) & (x_train_prep['delinq_2yrs'] < 4), 1, 0)
x_train_prep['delinq_2yrs:4_7'] = np.where((x_train_prep['delinq_2yrs'] >= 4) & (x_train_prep['delinq_2yrs'] < 7), 1, 0)
x_train_prep['delinq_2yrs:>7'] = np.where((x_train_prep['delinq_2yrs'] >= 7), 1, 0)

In [ ]:
list_of_dummy_variables.extend(["delinq_2yrs:0", "delinq_2yrs:1_3", "delinq_2yrs:4_7", "delinq_2yrs:>7"])
list_of_ref_categories.append("delinq_2yrs:>7")

In [ ]:
inq_last_6mths_woe = compute_woe_iv_continous(x_train_prep, "inq_last_6mths", y_train_prep)
inq_last_6mths_woe


In [ ]:
plot_woe(inq_last_6mths_woe, 90)

In [ ]:
x_train_prep['inq_last_6mths:0'] = np.where((x_train_prep['inq_last_6mths'] == 0), 1, 0)
x_train_prep['inq_last_6mths:1_2'] = np.where((x_train_prep['inq_last_6mths'] >= 1) & (x_train_prep['inq_last_6mths'] <= 2), 1, 0)
x_train_prep['inq_last_6mths:3_6'] = np.where((x_train_prep['inq_last_6mths'] >= 3) & (x_train_prep['inq_last_6mths'] <= 6), 1, 0)
x_train_prep['inq_last_6mths:>6'] = np.where((x_train_prep['inq_last_6mths'] > 6), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["inq_last_6mths:0", "inq_last_6mths:1_2", "inq_last_6mths:3_6", "inq_last_6mths:>6"])
    list_of_ref_categories.append("inq_last_6mths:>6")

In [ ]:
open_acc_woe = compute_woe_iv_continous(x_train_prep, "open_acc", y_train_prep)
open_acc_woe

In [ ]:
plot_woe(open_acc_woe, 90)

In [ ]:
plot_woe(open_acc_woe.loc[:40,:], 90)

In [ ]:
x_train_prep['open_acc:<4'] = np.where((x_train_prep['open_acc'] < 4), 1, 0)
x_train_prep['open_acc:4_12'] = np.where((x_train_prep['open_acc'] >= 4) & (x_train_prep['open_acc'] <= 12), 1, 0)
x_train_prep['open_acc:13_17'] = np.where((x_train_prep['open_acc'] >= 13) & (x_train_prep['open_acc'] <= 17), 1, 0)
x_train_prep['open_acc:18_22'] = np.where((x_train_prep['open_acc'] >= 18) & (x_train_prep['open_acc'] <= 22), 1, 0)
x_train_prep['open_acc:23_25'] = np.where((x_train_prep['open_acc'] >= 23) & (x_train_prep['open_acc'] <= 25), 1, 0)
x_train_prep['open_acc:26_30'] = np.where((x_train_prep['open_acc'] >= 26) & (x_train_prep['open_acc'] <= 30), 1, 0)
x_train_prep['open_acc:>=31'] = np.where((x_train_prep['open_acc'] >= 31), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["open_acc:<4", "open_acc:4_12", "open_acc:13_17", "open_acc:18_22", "open_acc:23_25", "open_acc:26_30", "open_acc:>=31" ])
    list_of_ref_categories.append("open_acc:<4")

In [ ]:
pub_rec_woe = compute_woe_iv_continous(x_train_prep, "pub_rec", y_train_prep)
pub_rec_woe

In [ ]:
plot_woe(pub_rec_woe)

In [ ]:
x_train_prep['pub_rec:0_2'] = np.where((x_train_prep['pub_rec'] >= 0) & (x_train_prep['pub_rec'] <= 2), 1, 0)
x_train_prep['pub_rec:3_4'] = np.where((x_train_prep['pub_rec'] >= 3) & (x_train_prep['pub_rec'] <= 4), 1, 0)
x_train_prep['pub_rec:>=5'] = np.where((x_train_prep['pub_rec'] >= 5), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["pub_rec:0_2", "pub_rec:3_4", "pub_rec:>=5"])
    list_of_ref_categories.append("pub_rec:0_2")

In [ ]:
x_train_prep['total_acc_factor'] = pd.cut(x_train_prep['total_acc'], 50)
total_acc_woe = compute_woe_iv_continous(x_train_prep, "total_acc_factor", y_train_prep)
total_acc_woe

In [ ]:
plot_woe(total_acc_woe, 90)

In [ ]:
x_train_prep['total_acc:<=28'] = np.where((x_train_prep['total_acc'] < 28), 1, 0)
x_train_prep['total_acc:28_50'] = np.where((x_train_prep['total_acc'] >= 28) & (x_train_prep['total_acc'] < 50), 1, 0)
x_train_prep['total_acc:>=50'] = np.where((x_train_prep['total_acc'] >= 50), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["total_acc:<=28", "total_acc:28_50", "total_acc:>=50"])
    list_of_ref_categories.append("total_acc:<=28")

In [ ]:
acc_now_delinq_woe = compute_woe_iv_continous(x_train_prep, "acc_now_delinq", y_train_prep)
acc_now_delinq_woe

In [ ]:
plot_woe(acc_now_delinq_woe, 90)

In [ ]:
x_train_prep['acc_now_delinq:0'] = np.where((x_train_prep['acc_now_delinq'] == 0), 1, 0)
x_train_prep['acc_now_delinq:>=1'] = np.where((x_train_prep['acc_now_delinq'] >= 1), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["acc_now_delinq:0", "acc_now_delinq:>=1"])
    list_of_ref_categories.append("acc_now_delinq:0")

In [ ]:
x_train_prep["dti_factor"] = pd.cut(x_train_prep["dti"], 50)
dti_woe = compute_woe_iv_continous(x_train_prep, "dti_factor", y_train_prep)
dti_woe

In [ ]:
plot_woe(dti_woe, 90)

In [ ]:
x_train_prep['dti:<=1.6'] = np.where((x_train_prep['dti'] < 1.6), 1, 0)
x_train_prep['dti:1.6_6.39'] = np.where((x_train_prep['dti'] >= 1.4) & (x_train_prep['dti'] <= 3.5), 1, 0)
x_train_prep['dti:6.39_10.39'] = np.where((x_train_prep['dti'] >= 6.39) & (x_train_prep['dti'] < 10.39), 1, 0)
x_train_prep['dti:12.97_16.79'] = np.where((x_train_prep['dti'] >= 12.97) & (x_train_prep['dti'] <= 16.79), 1, 0)
x_train_prep['dti:17.59_19.95'] = np.where((x_train_prep['dti'] >= 17.59) & (x_train_prep['dti'] <= 19.95), 1, 0)
x_train_prep['dti:20.75_23.99'] = np.where((x_train_prep['dti'] >= 20.75) & (x_train_prep['dti'] <= 23.99), 1, 0)
x_train_prep['dti:24.9_31.99'] = np.where((x_train_prep['dti'] >= 24.9) & (x_train_prep['dti'] <= 31.99), 1, 0)
x_train_prep['dti:>32'] = np.where((x_train_prep['dti'] > 32), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["dti:<=1.6", "dti:1.6_6.39", "dti:6.39_10.39", "dti:12.97_16.79", "dti:17.59_19.95", "dti:20.75_23.99", "dti:24.9_31.99", "dti:>32"])
    list_of_ref_categories.append("dti:>32")

In [ ]:
x_train_prep["mths_since_last_record_factor"] = pd.cut(x_train_prep["mths_since_last_record"], 50)
mths_since_last_record_woe = compute_woe_iv_continous(x_train_prep, "mths_since_last_record_factor", y_train_prep)
mths_since_last_record_woe

In [ ]:
plot_woe(mths_since_last_record_woe, 90)

In [ ]:
x_train_prep['mths_since_last_record:0_2'] = np.where((x_train_prep['mths_since_last_record'] < 2.5), 1, 0)
x_train_prep['mths_since_last_record:>2'] = np.where((x_train_prep['mths_since_last_record'] >= 2.5), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["mths_since_last_record:0_2", "mths_since_last_record:>2"])
    list_of_ref_categories.append("mths_since_last_record:0_2")

In [ ]:
x_train_prep["total_rev_hi_lim_factor"] = pd.cut(x_train_prep["total_rev_hi_lim"], 2000)
total_rev_woe = compute_woe_iv_continous(x_train_prep, "total_rev_hi_lim_factor", y_train_prep)
total_rev_woe

In [ ]:
plot_woe(total_rev_woe.iloc[: 50, : ],90)

In [ ]:
# '<=5K', '5K-10K', '10K-20K', '20K-30K', '30K-40K', '40K-55K', '55K-95K', '>95K'
x_train_prep['total_rev_hi_lim:<=5K'] = np.where((x_train_prep['total_rev_hi_lim'] <= 5000), 1, 0)
x_train_prep['total_rev_hi_lim:5K_10K'] = np.where((x_train_prep['total_rev_hi_lim'] > 5000) & (x_train_prep['total_rev_hi_lim'] <= 10000), 1, 0)
x_train_prep['total_rev_hi_lim:10K_20K'] = np.where((x_train_prep['total_rev_hi_lim'] > 10000) & (x_train_prep['total_rev_hi_lim'] <= 20000), 1, 0)
x_train_prep['total_rev_hi_lim:20K_30K'] = np.where((x_train_prep['total_rev_hi_lim'] > 20000) & (x_train_prep['total_rev_hi_lim'] <= 30000), 1, 0)
x_train_prep['total_rev_hi_lim:30K_40K'] = np.where((x_train_prep['total_rev_hi_lim'] > 30000) & (x_train_prep['total_rev_hi_lim'] <= 40000), 1, 0)
x_train_prep['total_rev_hi_lim:40K_55K'] = np.where((x_train_prep['total_rev_hi_lim'] > 40000) & (x_train_prep['total_rev_hi_lim'] <= 55000), 1, 0)
x_train_prep['total_rev_hi_lim:55K_95K'] = np.where((x_train_prep['total_rev_hi_lim'] > 55000) & (x_train_prep['total_rev_hi_lim'] <= 95000), 1, 0)
x_train_prep['total_rev_hi_lim:>95K'] = np.where((x_train_prep['total_rev_hi_lim'] > 95000), 1, 0)

In [ ]:
if input_data == "train":
    list_of_dummy_variables.extend(["total_rev_hi_lim:<=5K", "total_rev_hi_lim:5K_10K", "total_rev_hi_lim:10K_20K", "total_rev_hi_lim:20K_30K", 
                                    "total_rev_hi_lim:30K_40K", "total_rev_hi_lim:40K_55K", "total_rev_hi_lim:55K_95K", "total_rev_hi_lim:>95K"])
    list_of_ref_categories.append("total_rev_hi_lim:<=5K")

In [ ]:
list_of_ref_categories

In [ ]:
dataset = pd.concat([x_train_prep, y_train_prep], axis=1)
dataset.to_csv(f"./data/processed_{input_data}.csv", index=False)


if input_data == "train":
    cat_varibels = pd.DataFrame({"woe_vars": list_of_dummy_variables})
    ref_vars = pd.DataFrame({"ref_vars": list_of_ref_categories})
    cat_varibels.to_csv(f"./data/woe_cat_vars.csv", index=False)
    ref_vars.to_csv(f"./data/woe_ref_vars.csv", index=False)
    

In [ ]:
del data, x_train, x_train_prep, y_train, y_train_prep, x_test, y_test

In [4]:
from pandas_profiling import ProfileReport

/home/whiskey/miniconda3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_348203/2274191625.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [5]:
profile = ProfileReport(data)

In [6]:
profile.to_file("data_profile.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 13.61it/s]
